# Econometria Aplicada

## Heterocedasticidade



João Ricardo Costa Filho \\
[joaocostafilho.com](https://)

## A regressão linear múltipla

**Motivação**: \\

O que explica o comércio bilateral entre os países?

O exercício quantitativo desta aula é baseado em [Gravity in R: a short workshop](https://imedkrisna.github.io/gravity/).

### Dados

Vamos utilizar os seguintes conjuntos de dados:

* Gravity: "Data to estimate gravity equations: trade, GDP, population, trade agreements, etc. For all country pairs, 1948-2020."

Eles estão disponíveis neste [link](https://www.cepii.fr/CEPII/en/bdd_modele/bdd_modele.asp). Iremos baixar a versão para o R:

In [ ]:
url      <- "https://www.cepii.fr/DATA_DOWNLOAD/gravity/legacy/202202/Gravity_rds_V202202.zip"

In [ ]:
destfile <- "Gravity_rds_V202202.zip"

In [ ]:
download.file( url, destfile, mode = "wb")

In [ ]:
unzip( destfile )

Podemos verificar no diretório no qual estamos trabalhando que os dados foram baixados. Para isso, clique na 'pasta' no menu vertical que está à esquerda.

Façamos o upload da base de dados:

In [ ]:
gravity <- readRDS("Gravity_V202202.rds")

Faremos alguns filtros. O primeiro, diz respeito ao ano dos dados. Trabalhemos com 2019, antes da pandemia. Depois, filtraremos os códigos dos páises de origem e de destino, respectivamente. Finalmente, excluíremos dos dados referentes à situação na qual o país de origem e destino é o mesmo:

In [ ]:
library(dplyr)

In [ ]:
gravity2 <- gravity %>% filter( year == 2019 ) %>%
            filter( country_id_o!="IDN.1" )    %>%
            filter( country_id_d!="IDN.1" )    %>%
            filter( iso3_o!=iso3_d )

Obtenha as primeiras 10 linhas da base de dados 'gravity2':

Obtenha a estrutura dos dados da base 'gravity2':

Criemos dentro do dataframe 'gravity2' algumas variáveis:

In [ ]:
gravity2 <- gravity2 %>% mutate( lndist    = log( distw_harmonic ),      # Distância (média harmônica)
                                 lngdpo    = log( gdp_o ),               # PIB do país de origem
                                 lngdpd    = log( gdp_d ),               # PIB do país de destino
                                 lngdpco   = log( gdpcap_o ),            # PIB per capita do país de origem
                                 lngdpcd   = log( gdpcap_d ),            # PIB per capita do país de destino
                                 lntrade = log( 1 + tradeflow_baci )     # Fluxo de comércio entre dois países
                                 )

# Heterocedasticidade

Estime a regressão do log dos fluxos comerciais (lntrade) sobre os PIBs dos países de origem e destino, sobre o log da distância e sobre as dummies quando os países de origem ou de destino pertencem à OMC:

In [ ]:
reg <- lm( lntrade ~ lngdpo + lngdpd + lndist + wto_o + wto_d, data=gravity2 )

Crie um objeto com os resíduos da regressão:

Crie um dataframe que contenha três variáveis: 'lngdpo', 'lngdpd' (que estão no dataframe 'wage1') e 'e_hat'. Para isso, primeiro esxtraia a base de dados utilizada na regressão:

In [ ]:
# Dados utilizados na regressão:
model_data <- model.frame(reg)

Agora, crie o dataframe:

Faça uma dispersão entre o ln do PIB do país de origem (eixo x) e os resíduos da regressão:  

Faça uma dispersão entre o ln do PIB do país de destino (eixo x) e os resíduos da regressão:  

Faça uma dispersão entre o ln dda distância (eixo x) e os resíduos da regressão:

### Teste de Breusch-Pagan

Podemos definir a variância de $y$ como uma função das variáveis explicativas ($x_i)$:

\begin{equation*}
\sigma_i^2=E\left(e_i^2\right) = h\left( \alpha_0 + \alpha_1 x_{i 1}+ \alpha_2 x_{i 2}+\ldots+\alpha_s x_{i s}\right)
\end{equation*}

\begin{equation*}
\begin{aligned}
& \mathcal{H}_0: \alpha_1=\alpha_2=\ldots=\alpha_s=0 \\
& \mathcal{H}_a: \text{ pelo menos um } \alpha_i \neq 0
\end{aligned}
\end{equation*}

Este é um teste cuja estatística segue uma distribuição de $\chi^2$.

In [ ]:
install.packages("lmtest")

In [ ]:
library(lmtest)

In [ ]:
bptest( reg )

O que podemos concluir?

## Como "resolver" o problema

In [ ]:
install.packages("sandwich")

In [ ]:
library(sandwich)

In [ ]:
summary( reg )

In [ ]:
coeftest( reg,
          vcov = vcovHC( reg, "HC0") )   # Erros-padrão de White (1980). IMPORTANTE: funciona para amostras grandes

# Mínimos Quadrados Ponderados

Qual é a relação entre a renda e a riquiza líquida? Trabalhemos com o exemplo de Heiss (2020), p. 169.

\


Heiss, F. (2020). [Using R for Introductory Econometrics](https://www.urfie.net/downloads/PDF/URfIE_web.pdf). 2nd edition.

In [ ]:
install.packages("wooldridge")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
data(k401ksubs, package="wooldridge")

Estime a seguinte regressão por MQO:

$
\text{nettfa}_i = \beta_0 + \beta_1 \cdot \text{inc}_i + \beta_2 \cdot ( \text{age}_i - 25)^2 + \beta_3 \cdot \text{male}_i + \beta_4 \cdot \text{e401k}_i + u_i.
$

Agora, estime a regressão,

$
\text{nettfa}_i = \beta_0 + \beta_1 \cdot \text{inc}_i + \beta_2 \cdot ( \text{age}_i - 25)^2 + \beta_3 \cdot \text{male}_i + \beta_4 \cdot \text{e401k}_i + u_i,
$
mas com pesos $w_i = \frac{1}{\text{inc}_i}$.


In [ ]:
wlsreg <- lm(nettfa ~ inc + I((age-25)^2) + male + e401k, weight=1/inc, data=k401ksubs, subset=(fsize==1) )

In [ ]:
summary(wlsreg)